In [2]:
import numpy as np
import plotly.graph_objects as go
from dash import Dash, dcc, html, Input, Output
import dash
import time

# Domain parameters
nx, ny = 100, 50         
lx, ly = 2.0, 1.0       
dx = lx / (nx - 1)
dy = ly / (ny - 1)
x = np.linspace(0, lx, nx)
y = np.linspace(0, ly, ny)
X, Y = np.meshgrid(x, y)

# Obstacle: a circle 
obstacle_center = (0.5, 0.5)  
obstacle_radius = 0.1        
obstacle = (X - obstacle_center[0])**2 + (Y - obstacle_center[1])**2 < obstacle_radius**2

# Fluid and simulation parameters
U0 = 1.0      # constant inflow speed in x-direction
nu = 0.01     # kinematic viscosity
dt = 0.001    # time step size
nt = 2000     # increased total number of time steps for a longer simulation
frame_interval = 10  # store a frame every 10 time steps

# Initialize fields: u and v velocity and pressure p.
u = np.ones((ny, nx)) * U0   
v = np.zeros((ny, nx))
p = np.zeros((ny, nx))

# Impose the no-slip condition on the obstacle
u[obstacle] = 0
v[obstacle] = 0

frames = []

print("Starting simulation ...")
start_time = time.time()

for n in range(nt):
    un = u.copy()
    vn = v.copy()


    u[1:-1, 1:-1] = (un[1:-1, 1:-1] -
                     dt * (un[1:-1, 1:-1] * (un[1:-1, 1:-1] - un[1:-1, 0:-2]) / dx +
                           vn[1:-1, 1:-1] * (un[1:-1, 1:-1] - un[0:-2, 1:-1]) / dy) +
                     nu * dt * ((un[1:-1, 2:] - 2 * un[1:-1, 1:-1] + un[1:-1, 0:-2]) / dx**2 +
                                (un[2:, 1:-1] - 2 * un[1:-1, 1:-1] + un[0:-2, 1:-1]) / dy**2))
    
    v[1:-1, 1:-1] = (vn[1:-1, 1:-1] -
                     dt * (un[1:-1, 1:-1] * (vn[1:-1, 1:-1] - vn[1:-1, 0:-2]) / dx +
                           vn[1:-1, 1:-1] * (vn[1:-1, 1:-1] - vn[0:-2, 1:-1]) / dy) +
                     nu * dt * ((vn[1:-1, 2:] - 2 * vn[1:-1, 1:-1] + vn[1:-1, 0:-2]) / dx**2 +
                                (vn[2:, 1:-1] - 2 * vn[1:-1, 1:-1] + vn[0:-2, 1:-1]) / dy**2))

    # --- Boundary Conditions for velocity ---
    # Left boundary (inflow)
    u[:, 0] = U0
    v[:, 0] = 0
    # Right boundary (outflow): zero-gradient
    u[:, -1] = u[:, -2]
    v[:, -1] = v[:, -2]
    # Top and bottom boundaries: simple Neumann (copy from adjacent)
    u[0, :] = u[1, :]
    u[-1, :] = u[-2, :]
    v[0, :] = 0
    v[-1, :] = 0

    # Solve Pressure Poisson Equation 
    pn = p.copy()
    for it in range(50):
        p[1:-1, 1:-1] = (((pn[1:-1, 2:] + pn[1:-1, 0:-2]) * dy**2 +
                          (pn[2:, 1:-1] + pn[0:-2, 1:-1]) * dx**2) -
                         dx**2 * dy**2 / dt *
                         ((u[1:-1, 2:] - u[1:-1, 0:-2]) / (2 * dx) +
                          (v[2:, 1:-1] - v[0:-2, 1:-1]) / (2 * dy))
                         ) / (2 * (dx**2 + dy**2))
        pn = p.copy()
        # Pressure boundary conditions: dp/dn = 0
        p[:, -1] = p[:, -2]
        p[:, 0] = p[:, 1]
        p[0, :] = p[1, :]
        p[-1, :] = p[-2, :]


    u[1:-1, 1:-1] = u[1:-1, 1:-1] - dt / dx * (p[1:-1, 1:-1] - p[1:-1, 0:-2])
    v[1:-1, 1:-1] = v[1:-1, 1:-1] - dt / dy * (p[1:-1, 1:-1] - p[0:-2, 1:-1])

    # Re-apply obstacle (no-slip) condition
    u[obstacle] = 0
    v[obstacle] = 0

    # --- Save frame data every 'frame_interval' steps ---
    if n % frame_interval == 0:
        frames.append({
            'u': u.copy(),
            'v': v.copy(),
            'p': p.copy(),
            'n': n  # current time step
        })
        
end_time = time.time()
print(f"Simulation completed in {end_time - start_time:.2f} seconds.")
print(f"Total saved frames: {len(frames)}")


# Vorticity = dv/dx - du/dy
for frame in frames:
    u_frame = frame['u']
    v_frame = frame['v']
    dv_dx = np.gradient(v_frame, dx, axis=1)
    du_dy = np.gradient(u_frame, dy, axis=0)
    frame['vorticity'] = dv_dx - du_dy


app = Dash(__name__)
server = app.server  

app.layout = html.Div([
    html.H1("2D Navier–Stokes: Flow Past a Circular Obstacle"),
    dcc.Graph(id='flow-graph'),
    html.Br(),
    dcc.Slider(
        id='time-slider',
        min=0,
        max=len(frames) - 1,
        value=0,
        marks={i: f"{frames[i]['n']}" for i in range(0, len(frames), max(1, len(frames)//10))},
        step=1
    ),
    dcc.Interval(
        id='interval',
        interval=200,  # in milliseconds
        n_intervals=0
    )
])

@app.callback(
    Output('flow-graph', 'figure'),
    Input('time-slider', 'value')
)
def update_graph(frame_index):
    frame = frames[frame_index]
    vort = frame['vorticity']
    fig = go.Figure(data=go.Heatmap(
        z=vort,
        x=x,
        y=y,
        colorscale='Viridis',
        colorbar=dict(title="Vorticity")
    ))
    fig.update_layout(
        title=f"Vorticity Field at Time Step {frame['n']}",
        xaxis_title="X",
        yaxis_title="Y",
        yaxis=dict(scaleanchor="x", scaleratio=1)
    )
    # Overlay the circular obstacle
    theta = np.linspace(0, 2 * np.pi, 100)
    circle_x = obstacle_center[0] + obstacle_radius * np.cos(theta)
    circle_y = obstacle_center[1] + obstacle_radius * np.sin(theta)
    fig.add_trace(go.Scatter(
        x=circle_x,
        y=circle_y,
        mode='lines',
        line=dict(color='red', width=3),
        name='Obstacle'
    ))
    return fig

@app.callback(
    Output('time-slider', 'value'),
    [Input('interval', 'n_intervals'),
     Input('time-slider', 'value')]
)
def animate_slider(n_intervals, current_value):
    new_val = (current_value + 1) % len(frames)
    return new_val


if __name__ == '__main__':
    app.run_server(debug=True, use_reloader=False)


Starting simulation ...
Simulation completed in 4.68 seconds.
Total saved frames: 200
